In [16]:
# NBA API Endpoint Testing Notebook
import nba_api.stats.endpoints as nbaapi
import rdshelp
import pandas as pd
import sys
import os
import json
from datetime import datetime, timedelta

# Add project paths - fix path resolution
current_dir = os.path.dirname(os.path.abspath(''))
if 'notebooks' in current_dir:
    # If we're in the notebooks directory, go up one level
    project_root = os.path.dirname(current_dir)
else:
    # If we're already in the project root
    project_root = current_dir

# Alternative approach - find the project root by looking for key files
def find_project_root():
    """Find the project root by looking for characteristic files"""
    current = os.path.abspath('')
    
    # Keep going up directories until we find the project root
    while current != os.path.dirname(current):  # Not at filesystem root
        if os.path.exists(os.path.join(current, 'endpoints', 'config', 'nba_endpoints_config.py')):
            return current
        current = os.path.dirname(current)
    
    # Fallback to hardcoded path
    return r'C:\Users\ajwin\Projects\Personal\thebigone'

project_root = find_project_root()
endpoints_path = os.path.join(project_root, 'endpoints')
endpoints_config_path = os.path.join(endpoints_path, 'config')

# Add to Python path
paths_to_add = [project_root, endpoints_path, endpoints_config_path]
for path in paths_to_add:
    if path not in sys.path:
        sys.path.insert(0, path)

print(f"Project root found: {project_root}")
print(f"Endpoints path: {endpoints_path}")
print(f"Config path: {endpoints_config_path}")

# Test if we can find the config file
config_file = os.path.join(endpoints_config_path, 'nba_endpoints_config.py')
if os.path.exists(config_file):
    print(f"✅ Config file found: {config_file}")
else:
    print(f"❌ Config file not found: {config_file}")
    print("Available files in endpoints/config:")
    if os.path.exists(endpoints_config_path):
        for file in os.listdir(endpoints_config_path):
            print(f"  - {file}")

# Import the configuration
try:
    from nba_endpoints_config import (
        ALL_ENDPOINTS, 
        get_endpoint_by_name, 
        list_all_endpoint_names,
        get_endpoints_by_priority
    )
    
    print("NBA API Endpoint Testing Setup Complete!")
    print(f"Available endpoints: {len(list_all_endpoint_names())}")
    print(f"High priority endpoints: {len(get_endpoints_by_priority('high'))}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Trying alternative import...")
    
    try:
        from endpoints.config.nba_endpoints_config import (
            ALL_ENDPOINTS, 
            get_endpoint_by_name, 
            list_all_endpoint_names,
            get_endpoints_by_priority
        )
        print("✅ Alternative import successful!")
        print(f"Available endpoints: {len(list_all_endpoint_names())}")
        print(f"High priority endpoints: {len(get_endpoints_by_priority('high'))}")
        
    except ImportError as e2:
        print(f"❌ Alternative import also failed: {e2}")
        print("Please check that the nba_endpoints_config.py file exists in the endpoints/config directory")

Project root found: c:\Users\ajwin\Projects\Personal\thebigone
Endpoints path: c:\Users\ajwin\Projects\Personal\thebigone\endpoints
Config path: c:\Users\ajwin\Projects\Personal\thebigone\endpoints\config
✅ Config file found: c:\Users\ajwin\Projects\Personal\thebigone\endpoints\config\nba_endpoints_config.py
NBA API Endpoint Testing Setup Complete!
Available endpoints: 41
High priority endpoints: 16


In [17]:
# Endpoint Selection and Testing Utilities
def list_endpoints_by_category():
    """Display all endpoints organized by category"""
    for category, endpoints in ALL_ENDPOINTS.items():
        print(f"\n=== {category.upper().replace('_', ' ')} ENDPOINTS ===")
        for i, endpoint in enumerate(endpoints, 1):
            priority_icon = "🔥" if endpoint['priority'] == 'high' else "📊" if endpoint['priority'] == 'medium' else "📝"
            print(f"{i:2d}. {priority_icon} {endpoint['endpoint']} - {endpoint['description']}")
            print(f"     Parameters: {endpoint['parameters']}")
            print(f"     Priority: {endpoint['priority']}, Frequency: {endpoint['frequency']}")

def select_endpoint(endpoint_name=None):
    """Select an endpoint for testing"""
    if endpoint_name is None:
        print("Available endpoints:")
        names = list_all_endpoint_names()
        for i, name in enumerate(names, 1):
            config = get_endpoint_by_name(name)
            priority_icon = "🔥" if config['priority'] == 'high' else "📊" if config['priority'] == 'medium' else "📝"
            print(f"{i:2d}. {priority_icon} {name}")
        
        choice = input(f"\nEnter endpoint name or number (1-{len(names)}): ").strip()
        
        if choice.isdigit():
            idx = int(choice) - 1
            if 0 <= idx < len(names):
                endpoint_name = names[idx]
            else:
                print("Invalid selection")
                return None
        else:
            endpoint_name = choice
    
    config = get_endpoint_by_name(endpoint_name)
    if config:
        print(f"\n✅ Selected: {endpoint_name}")
        print(f"   Description: {config['description']}")
        print(f"   Parameters: {config['parameters']}")
        print(f"   Priority: {config['priority']}")
        return config
    else:
        print(f"❌ Endpoint '{endpoint_name}' not found")
        return None

# Quick endpoint lookup
def quick_test_endpoint(endpoint_name):
    """Quick function to test a specific endpoint"""
    config = get_endpoint_by_name(endpoint_name)
    if not config:
        print(f"❌ Endpoint '{endpoint_name}' not found")
        return
    
    print(f"Testing {endpoint_name}...")
    resolved_params = resolve_test_parameters(config)
    if resolved_params:
        result = test_api_call(endpoint_name, resolved_params)
        return result
    else:
        print("❌ Could not resolve parameters")
        return None

# Display available endpoints
list_endpoints_by_category()


=== GAME BASED ENDPOINTS ===
 1. 🔥 BoxScoreAdvancedV3 - Advanced box score stats V3 (PREFERRED)
     Parameters: {'game_id': 'from_mastergames'}
     Priority: high, Frequency: after_game_completion
 2. 📊 BoxScoreAdvancedV2 - Advanced box score stats for a game (backup)
     Parameters: {'game_id': 'from_mastergames'}
     Priority: medium, Frequency: after_game_completion
 3. 📊 BoxScoreDefensiveV2 - Defensive stats for a game
     Parameters: {'game_id': 'from_mastergames'}
     Priority: medium, Frequency: after_game_completion
 4. 🔥 BoxScoreFourFactorsV3 - Four factors stats V3 (PREFERRED)
     Parameters: {'game_id': 'from_mastergames'}
     Priority: high, Frequency: after_game_completion
 5. 📊 BoxScoreFourFactorsV2 - Four factors stats for a game (backup)
     Parameters: {'game_id': 'from_mastergames'}
     Priority: medium, Frequency: after_game_completion
 6. 📝 BoxScoreHustleV2 - Hustle stats for a game
     Parameters: {'game_id': 'from_mastergames'}
     Priority: low, Freq

In [18]:
# Parameter Resolution and API Testing Functions
def get_current_season():
    """Get current NBA season string"""
    now = datetime.now()
    if now.month >= 10:  # Season starts in October
        return f"{now.year}-{str(now.year + 1)[2:]}"
    else:
        return f"{now.year - 1}-{str(now.year)[2:]}"

def resolve_test_parameters(endpoint_config):
    """Resolve endpoint parameters for testing with sample data"""
    parameters = endpoint_config.get('parameters', {})
    resolved_params = {}
    
    # Sample data for testing
    SAMPLE_DATA = {
        'game_id': '0022400001',    # Recent NBA game
        'player_id': 2544,         # LeBron James
        'team_id': 1610612747,     # Los Angeles Lakers
        'vs_player_id': 201939,    # Stephen Curry
        'last_n_games': 10,
        'season': get_current_season(),
        'date_from': (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'),
        'date_to': datetime.now().strftime('%Y-%m-%d'),
        'league_id_nullable': '00',
        'season_nullable': get_current_season(),
        'season_type_nullable': 'Regular Season'
    }
    
    print(f"Resolving parameters for {endpoint_config['endpoint']}...")
    
    for param_key, param_source in parameters.items():
        print(f"  Parameter: {param_key} (source: {param_source})")
        
        # Handle different parameter source types
        if isinstance(param_source, str):
            if param_source == 'from_mastergames':
                resolved_params[param_key] = SAMPLE_DATA['game_id']
                print(f"    ✅ Resolved to sample game_id: {resolved_params[param_key]}")
                
            elif param_source == 'from_masterplayers':
                resolved_params[param_key] = SAMPLE_DATA['player_id']
                print(f"    ✅ Resolved to sample player_id: {resolved_params[param_key]}")
                
            elif param_source == 'from_masterteams':
                resolved_params[param_key] = SAMPLE_DATA['team_id']
                print(f"    ✅ Resolved to sample team_id: {resolved_params[param_key]}")
                
            elif param_source == 'current_season':
                resolved_params[param_key] = SAMPLE_DATA['season']
                print(f"    ✅ Resolved to current season: {resolved_params[param_key]}")
                
            elif param_source == 'dynamic_date_range':
                if 'from' in param_key.lower():
                    resolved_params[param_key] = SAMPLE_DATA['date_from']
                elif 'to' in param_key.lower():
                    resolved_params[param_key] = SAMPLE_DATA['date_to']
                else:
                    resolved_params[param_key] = SAMPLE_DATA['date_from']  # Default
                print(f"    ✅ Resolved to date: {resolved_params[param_key]}")
                
            elif param_source == 'dynamic':
                # Use sample data based on parameter name
                if 'player' in param_key:
                    resolved_params[param_key] = SAMPLE_DATA.get('vs_player_id', SAMPLE_DATA['player_id'])
                elif 'team' in param_key:
                    resolved_params[param_key] = SAMPLE_DATA['team_id']
                else:
                    resolved_params[param_key] = 'dynamic_value'
                print(f"    ✅ Resolved dynamic parameter: {resolved_params[param_key]}")
                
            else:
                # It's a literal string value
                resolved_params[param_key] = param_source
                print(f"    ✅ Using literal value: {resolved_params[param_key]}")
                
        elif isinstance(param_source, (int, float, bool)):
            # Static numeric or boolean value
            resolved_params[param_key] = param_source
            print(f"    ✅ Using static value: {resolved_params[param_key]}")
            
        else:
            print(f"    ⚠️  Unknown parameter source type: {type(param_source)}")
            resolved_params[param_key] = str(param_source)
    
    return resolved_params

def test_api_call(endpoint_name, parameters):
    """Test an API call with the given parameters"""
    try:
        print(f"\n🔄 Testing API call: {endpoint_name}")
        print(f"📋 Parameters: {parameters}")
        
        # Get the endpoint class
        endpoint_class = getattr(nbaapi, endpoint_name)
        print(f"✅ Found endpoint class: {endpoint_class}")
        
        # Make the API call
        print("🌐 Making API call...")
        endpoint_instance = endpoint_class(**parameters)
        dataframes = endpoint_instance.get_data_frames()
        
        if dataframes is None:
            print("⚠️  API returned None")
            return None
            
        print(f"✅ API call successful!")
        print(f"📊 Returned {len(dataframes)} dataframes")
        
        # Analyze the dataframes
        for i, df in enumerate(dataframes):
            if df is not None and hasattr(df, 'shape'):
                print(f"   Dataframe {i}: {df.shape[0]} rows × {df.shape[1]} columns")
                if not df.empty:
                    print(f"      Sample columns: {list(df.columns)[:5]}")
                    print(f"      Data types: {len(df.dtypes.unique())} unique types")
            else:
                print(f"   Dataframe {i}: Empty or None")
        
        return dataframes
        
    except Exception as e:
        error_msg = str(e)
        print(f"❌ API call failed: {error_msg}")
        
        # Classify the error
        if any(indicator in error_msg.lower() for indicator in ['invalid', 'parameter', 'required']):
            print("🔍 This appears to be a parameter error")
        elif 'nonetype' in error_msg.lower() or 'keys' in error_msg.lower():
            print("🔍 This appears to be an NBA API data issue")
        else:
            print("🔍 This appears to be a system/network error")
            
        return None

print("Parameter resolution and testing functions loaded!")
print(f"Current NBA season: {get_current_season()}")

Parameter resolution and testing functions loaded!
Current NBA season: 2024-25


# Interactive Endpoint Testing

Use the functions above to test NBA API endpoints:

## Quick Testing
- `quick_test_endpoint('BoxScoreTraditionalV3')` - Test a specific endpoint
- `select_endpoint()` - Interactive endpoint selection
- `list_endpoints_by_category()` - Show all available endpoints

## Step-by-Step Testing
1. Select an endpoint: `config = select_endpoint('EndpointName')`
2. Resolve parameters: `params = resolve_test_parameters(config)`
3. Test the API call: `result = test_api_call('EndpointName', params)`

## Examples Ready to Run
Try these in the next cell!

In [19]:
# Example 1: Test the BoxScoreTraditionalV3 endpoint (the one you have selected)
print("=== Testing BoxScoreTraditionalV3 ===")
result = quick_test_endpoint('BoxScoreTraditionalV3')

if result:
    print(f"\n📊 Success! Got {len(result)} dataframes")
    for i, df in enumerate(result):
        if df is not None and not df.empty:
            print(f"Dataframe {i+1}: {df.shape}")
            print(f"  Sample data:")
            print(df.head(2))
            print()
else:
    print("❌ Test failed")

=== Testing BoxScoreTraditionalV3 ===
Testing BoxScoreTraditionalV3...
Resolving parameters for BoxScoreTraditionalV3...
  Parameter: game_id (source: from_mastergames)
    ✅ Resolved to sample game_id: 0022400001

🔄 Testing API call: BoxScoreTraditionalV3
📋 Parameters: {'game_id': '0022400001'}
✅ Found endpoint class: <class 'nba_api.stats.endpoints.boxscoretraditionalv3.BoxScoreTraditionalV3'>
🌐 Making API call...
✅ API call successful!
📊 Returned 3 dataframes
   Dataframe 0: 25 rows × 34 columns
      Sample columns: ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode']
      Data types: 3 unique types
   Dataframe 1: 4 rows × 26 columns
      Sample columns: ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode']
      Data types: 3 unique types
   Dataframe 2: 2 rows × 26 columns
      Sample columns: ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode']
      Data types: 3 unique types

📊 Success! Got 3 dataframes
Dataframe 1: (25, 34)
  Sample data:
       gameId

In [20]:
# Example 2: Test multiple high-priority endpoints
print("=== Testing High Priority Game Endpoints ===")

high_priority_game_endpoints = [
    'BoxScoreAdvancedV3', 
    'BoxScoreFourFactorsV3', 
    'BoxScoreMiscV3',
    'BoxScoreTraditionalV3',
    'BoxScoreUsageV3',
    'PlayByPlayV3'
]

results = {}

for endpoint_name in high_priority_game_endpoints:
    print(f"\n--- Testing {endpoint_name} ---")
    try:
        config = get_endpoint_by_name(endpoint_name)
        if config:
            params = resolve_test_parameters(config)
            result = test_api_call(endpoint_name, params)
            results[endpoint_name] = {
                'success': result is not None,
                'dataframes': len(result) if result else 0,
                'config': config
            }
        else:
            results[endpoint_name] = {'success': False, 'error': 'Config not found'}
            
    except Exception as e:
        results[endpoint_name] = {'success': False, 'error': str(e)}
        print(f"❌ Exception: {e}")

# Summary
print(f"\n{'='*50}")
print("ENDPOINT TESTING SUMMARY")
print(f"{'='*50}")

successful = 0
total = len(results)

for endpoint, result in results.items():
    status = "✅ SUCCESS" if result['success'] else "❌ FAILED"
    dataframes = f"({result.get('dataframes', 0)} dataframes)" if result['success'] else ""
    print(f"{status:<12} {endpoint:<25} {dataframes}")
    if result['success']:
        successful += 1

print(f"\nOverall Success Rate: {successful}/{total} ({successful/total*100:.1f}%)")
print("="*50)

=== Testing High Priority Game Endpoints ===

--- Testing BoxScoreAdvancedV3 ---
Resolving parameters for BoxScoreAdvancedV3...
  Parameter: game_id (source: from_mastergames)
    ✅ Resolved to sample game_id: 0022400001

🔄 Testing API call: BoxScoreAdvancedV3
📋 Parameters: {'game_id': '0022400001'}
✅ Found endpoint class: <class 'nba_api.stats.endpoints.boxscoreadvancedv3.BoxScoreAdvancedV3'>
🌐 Making API call...
✅ API call successful!
📊 Returned 2 dataframes
   Dataframe 0: 25 rows × 37 columns
      Sample columns: ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode']
      Data types: 3 unique types
   Dataframe 1: 2 rows × 30 columns
      Sample columns: ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode']
      Data types: 3 unique types

--- Testing BoxScoreFourFactorsV3 ---
Resolving parameters for BoxScoreFourFactorsV3...
  Parameter: game_id (source: from_mastergames)
    ✅ Resolved to sample game_id: 0022400001

🔄 Testing API call: BoxScoreFourFactorsV3
📋 Param

In [ ]:
# Custom Endpoint Testing - Choose Your Own!

# Option 1: Interactive selection (uncomment to use)
# selected_config = select_endpoint()
# if selected_config:
#     params = resolve_test_parameters(selected_config)
#     result = test_api_call(selected_config['endpoint'], params)

# Option 2: Test a specific endpoint (edit the endpoint name below)
endpoint_to_test = 'BoxScoreTraditionalV3'  # Change this to test different endpoints

print(f"🎯 Custom Testing: {endpoint_to_test}")
print("="*50)

# Get the configuration
config = get_endpoint_by_name(endpoint_to_test)

if config:
    print(f"📝 Endpoint: {config['endpoint']}")
    print(f"📖 Description: {config['description']}")
    print(f"⚡ Priority: {config['priority']}")
    print(f"🔄 Frequency: {config['frequency']}")
    print(f"📋 Parameters: {config['parameters']}")
    
    print(f"\n🔧 Resolving parameters...")
    resolved_params = resolve_test_parameters(config)
    
    if resolved_params:
        print(f"\n🚀 Making API call...")
        result = test_api_call(endpoint_to_test, resolved_params)
        
        if result:
            print(f"\n🎉 SUCCESS! Detailed analysis:")
            
            # Detailed dataframe analysis
            for i, df in enumerate(result):
                if df is not None and not df.empty:
                    print(f"\n📊 Dataframe {i+1}:")
                    print(f"   Shape: {df.shape}")
                    print(f"   Columns: {list(df.columns)}")
                    print(f"   Data types: {df.dtypes.to_dict()}")
                    print(f"   Sample data:")
                    print(df.head(3).to_string())
                    
                    # Check for key basketball stats
                    basketball_keywords = ['pts', 'reb', 'ast', 'fg', 'ft', 'player', 'team', 'min']
                    found_keywords = [col for col in df.columns if any(keyword in col.lower() for keyword in basketball_keywords)]
                    if found_keywords:
                        print(f"   Basketball stats found: {found_keywords}")
                else:
                    print(f"\n📊 Dataframe {i+1}: Empty or None")
    else:
        print("❌ Could not resolve parameters")
else:
    print(f"❌ Endpoint '{endpoint_to_test}' not found in configuration")
    print("Available endpoints:")
    for name in list_all_endpoint_names()[:10]:  # Show first 10
        print(f"  - {name}")
    print("  ... and more")